In [1]:
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
python.exe -m pip install --upgrade pip

SyntaxError: invalid syntax (842801469.py, line 1)

In [3]:
from dotenv import load_dotenv

In [4]:
load_dotenv()

True

In [5]:
from openai import OpenAI

openai_client = OpenAI()

In [6]:
!uv add openai-agents==0.3.3

Resolved 221 packages in 3ms
Audited 201 packages in 913ms


In [15]:
!pip install openai requests beautifulsoup4 --break-system-packages


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
!uv remove openai

error: The dependency `openai` could not be found in `project.dependencies`


In [8]:
import agents

In [9]:
agents.__version__

'0.3.3'

In [10]:
from agents import Agent, function_tool, Runner

In [11]:
from agents import Runner

In [12]:
runner = Runner()

In [13]:
from toyaikit.chat import IPythonChatInterface
from toyaikit.chat.runners import OpenAIAgentsSDKRunner

chat_interface = IPythonChatInterface()




In [14]:
runner = OpenAIAgentsSDKRunner(
    chat_interface=chat_interface,
    agent=web_agent
)

NameError: name 'web_agent' is not defined

In [33]:
import requests
from bs4 import BeautifulSoup

def fetch_web_page(url: str) -> str:
    """
    Fetch the content of a web page given its URL.
    """
    try:
        # Add headers to avoid being blocked
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        
        # Send GET request with headers
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        
        # Parse HTML content
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Remove script and style elements
        for script in soup(["script", "style"]):
            script.decompose()
        
        # Get text content
        text = soup.get_text()
        
        # Clean up whitespace
        lines = (line.strip() for line in text.splitlines())
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        text = ' '.join(chunk for chunk in chunks if chunk)
        
        return text
    
    except Exception as e:
        return f"Error fetching web page: {str(e)}"

In [17]:
from openai import OpenAI
import json

# Initialize OpenAI client
client = OpenAI()

# Define the tool for the agent
tools = [
    {
        "type": "function",
        "function": {
            "name": "fetch_web_page",
            "description": "Fetch the content of a web page given its URL",
            "parameters": {
                "type": "object",
                "properties": {
                    "url": {
                        "type": "string",
                        "description": "The URL of the web page to fetch"
                    }
                },
                "required": ["url"]
            }
        }
    }
]

def run_agent(user_message: str):
    """Run the agent with web fetching capability."""
    messages = [{"role": "user", "content": user_message}]
    
    # Initial API call
    response = client.chat.completions.create(
        model="gpt-4",
        messages=messages,
        tools=tools,
        tool_choice="auto"
    )
    
    response_message = response.choices[0].message
    messages.append(response_message)
    
    # Handle tool calls
    if response_message.tool_calls:
        for tool_call in response_message.tool_calls:
            if tool_call.function.name == "fetch_web_page":
                function_args = json.loads(tool_call.function.arguments)
                url = function_args.get("url")
                
                print(f"🔍 Fetching: {url}")
                
                # Call our function
                function_response = fetch_web_page(url)
                
                # Add function response to messages
                messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "content": function_response
                })
        
        # Get final response
        final_response = client.chat.completions.create(
            model="gpt-4",
            messages=messages
        )
        return final_response.choices[0].message.content
    
    return response_message.content

In [18]:
result = run_agent("Can you fetch the content from https://example.com and tell me what it says?")


🔍 Fetching: https://example.com


In [19]:
print(result)

The website at https://example.com is an 'Example Domain' and is intended to be used in documentation examples without needing permission. It is suggested to avoid use in operations. You can learn more on their website.


In [ ]:
##Summary Agent

In [20]:
import os
from datetime import datetime

In [27]:
def save_summary(content: str, filename: str = None) -> str:
    """
    Saves a summary to a text file in the summaries directory.
    
    Args:
        content (str): The content to save.
        filename (str, optional): The filename. If not provided, uses timestamp.
    
    Returns:
        str: The saved content and confirmation message.
    """
    try:
        # Create summaries directory if it doesn't exist
        os.makedirs("summaries", exist_ok=True)
        
        # Generate filename if not provided
        if filename is None:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"summary_{timestamp}.txt"
        
        # Ensure .txt extension
        if not filename.endswith('.txt'):
            filename += '.txt'
        
        filepath = os.path.join("summaries", filename)
        
        # Write content to file
        with open(filepath, 'w', encoding='utf-8') as f:
            f.write(content)
        
        # Return both the content and confirmation
        return f"Summary saved successfully to {filepath}\n\n--- SAVED CONTENT ---\n{content}"
    except Exception as e:
        return f"Error saving summary: {str(e)}"

In [22]:
# Add save_summary tool to your existing tools list
tools = [
    {
        "type": "function",
        "function": {
            "name": "fetch_web_page",
            "description": "Fetch the content of a web page given its URL",
            "parameters": {
                "type": "object",
                "properties": {
                    "url": {
                        "type": "string",
                        "description": "The URL of the web page to fetch"
                    }
                },
                "required": ["url"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "save_summary",
            "description": "Save a summary to a text file",
            "parameters": {
                "type": "object",
                "properties": {
                    "content": {
                        "type": "string",
                        "description": "The content to save"
                    },
                    "filename": {
                        "type": "string",
                        "description": "Optional filename for the summary. If not provided, a timestamp will be used."
                    }
                },
                "required": ["content"]
            }
        }
    }
]

In [30]:
def run_agent(user_message: str):
    """Run the agent with web fetching and save summary capabilities."""
    
    # Add system message with instructions
    messages = [
        {
            "role": "system", 
            "content": """You are a helpful web assistant. When a user asks "What is this page about?" or similar questions about a webpage:
1. First, use the fetch_web_page tool to get the content
2. Then, create a clear summary of the page
3. Finally, use the save_summary tool to save your summary
4. Show the user what you found and confirm the summary was saved"""
        },
        {"role": "user", "content": user_message}
    ]
    
    # Initial API call
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=tools,
        tool_choice="auto"
    )
    
    response_message = response.choices[0].message
    messages.append(response_message)
    
    # Handle tool calls
    while response_message.tool_calls:
        for tool_call in response_message.tool_calls:
            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments)
            
            if function_name == "fetch_web_page":
                url = function_args.get("url")
                print(f"🔍 Fetching: {url}")
                function_response = fetch_web_page(url)
                
            elif function_name == "save_summary":
                content = function_args.get("content")
                filename = function_args.get("filename")
                print(f"💾 Saving summary...")
                function_response = save_summary(content, filename)
            
            # Add function response to messages
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": function_response
            })
        
        # Get next response
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            tools=tools,
            tool_choice="auto"
        )
        response_message = response.choices[0].message
        messages.append(response_message)
    
    return response_message.content

In [34]:
# Test it: Fetch a page, summarize it, and save
result = run_agent("What is this page about? https://en.wikipedia.org/wiki/Capybara")

🔍 Fetching: https://en.wikipedia.org/wiki/Capybara
💾 Saving summary...


In [35]:
print(result)

I found detailed information on the Wikipedia page about capybaras, which are the largest rodents in the world, scientifically known as *Hydrochoerus hydrochaeris*. They are native to South America and typically inhabit areas near water bodies such as savannas and forests. 

The page covers various aspects of capybaras, including their classification, physical description, ecology, social behavior, reproduction, and their interactions with humans. Capybaras are social animals, often living in groups, and primarily feed on grasses and aquatic plants. They are not considered endangered and have stable populations. Additionally, the article highlights their cultural significance and current popularity in media and social media.

I have saved this summary successfully. If you need further information or details, feel free to ask!


In [ ]:
##Search Agent

In [36]:
from minsearch import AppendableIndex

In [37]:
index = AppendableIndex(text_fields=['content'])

In [38]:
import docs

In [58]:
# Improve the search function to handle synonyms and related terms
class SearchTools:
    """Tools for searching web content."""
    
    def __init__(self):
        self.indexed_pages = {}  # Store indexed content: {url: content}
    
    def index_page(self, url: str, content: str = None) -> bool:
        """
        Index a web page for searching.
        
        Args:
            url (str): The URL of the page to index
            content (str): Optional pre-fetched content. If None, will fetch it.
            
        Returns:
            bool: True if indexing succeeds, False otherwise.
        """
        try:
            if not content:
                # Fetch the page content
                content = fetch_web_page(url)
                if content.startswith("Error"):
                    print(f"Failed to fetch page: {url}")
                    return False
            
            # Store the indexed content
            self.indexed_pages[url] = content
            print(f"✅ Successfully indexed: {url}")
            return True
            
        except Exception as e:
            print(f"Error indexing page {url}: {e}")
            return False
    
    def search(self, query: str) -> str:
        """
        Search through indexed pages for relevant content.
        
        Args:
            query (str): The search query
            
        Returns:
            str: Search results
        """
        if not isinstance(query, str) or not query.strip():
            return "Error: Query must be a non-empty string."
        
        if not self.indexed_pages:
            return "No pages have been indexed yet. Please index a page first using index_page."
        
        try:
            results = []
            query_lower = query.lower()
            
            # Extract keywords from the query
            keywords = [word.strip('?,!.') for word in query_lower.split() if len(word) > 3]
            
            # Search for pages containing the keywords
            for url, content in self.indexed_pages.items():
                content_lower = content.lower()
                matches = []
                
                # Find all occurrences of keywords
                for keyword in keywords:
                    if keyword in content_lower:
                        # Find context around the keyword
                        start = 0
                        while True:
                            index = content_lower.find(keyword, start)
                            if index == -1:
                                break
                            
                            # Get snippet around the keyword
                            snippet_start = max(0, index - 150)
                            snippet_end = min(len(content), index + 150)
                            snippet = content[snippet_start:snippet_end].strip()
                            
                            matches.append(snippet)
                            start = index + 1
                            
                            # Limit to 2 matches per keyword per page
                            if len(matches) >= 2:
                                break
                        
                        if len(matches) >= 3:
                            break
                
                if matches:
                    results.append({
                        'url': url,
                        'snippets': matches[:3]  # Limit to 3 snippets per page
                    })
            
            if not results:
                return f"No results found for query: '{query}'. Try different keywords or index more pages."
            
            # Format results
            output = f"Found {len(results)} page(s) with relevant information for '{query}':\n\n"
            for i, result in enumerate(results, 1):
                output += f"{i}. URL: {result['url']}\n"
                for j, snippet in enumerate(result['snippets'], 1):
                    output += f"   Match {j}: ...{snippet}...\n"
                output += "\n"
            
            return output
            
        except Exception as e:
            return f"Error performing search: {e}"

In [59]:
search_tools = SearchTools()

In [60]:
search_agent_tools = [
    {
        "type": "function",
        "function": {
            "name": "index_page",
            "description": "Index a web page so it can be searched later",
            "parameters": {
                "type": "object",
                "properties": {
                    "url": {
                        "type": "string",
                        "description": "The URL of the page to index"
                    },
                    "content": {
                        "type": "string",
                        "description": "Optional pre-fetched content"
                    }
                },
                "required": ["url"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "search",
            "description": "Search through indexed pages for relevant information",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The search query (natural language, NOT a URL)"
                    }
                },
                "required": ["query"]
            }
        }
    }
]

In [61]:
# Update the search agent runner with better instructions
def run_search_agent(user_message: str):
    """Run the search agent."""
    
    search_instructions = """
Your task is to search through indexed web pages.

Before performing a search:
1. Check if relevant pages have been indexed
2. If pages are already indexed, proceed directly to searching
3. If no pages are indexed, ask the user to provide a URL to index first

Important rules:
- Do NOT call `search` using a URL or link as the query
- After successfully indexing a page, do NOT perform a search automatically unless the user explicitly asks
- When the user asks a question (not about indexing), use the `search` tool with their question
- Only use natural-language queries for search (e.g., "threats to capybara populations")

If pages are already indexed and the user asks a question, search immediately without asking for more URLs.
"""
    
    messages = [
        {"role": "system", "content": search_instructions},
        {"role": "user", "content": user_message}
    ]
    
    # Add context about indexed pages
    if search_tools.indexed_pages:
        indexed_urls = "\n".join([f"- {url}" for url in search_tools.indexed_pages.keys()])
        messages.insert(1, {
            "role": "system",
            "content": f"Currently indexed pages:\n{indexed_urls}\n\nYou can search these pages directly."
        })
    
    response = client.chat.completions.create(
        model="gpt-4",
        messages=messages,
        tools=search_agent_tools,
        tool_choice="auto"
    )
    
    response_message = response.choices[0].message
    messages.append(response_message)
    
    # Handle tool calls
    while response_message.tool_calls:
        for tool_call in response_message.tool_calls:
            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments)
            
            if function_name == "index_page":
                url = function_args.get("url")
                content = function_args.get("content")
                print(f"📚 Indexing page: {url}")
                success = search_tools.index_page(url, content)
                function_response = f"Page indexed successfully: {success}"
                
            elif function_name == "search":
                query = function_args.get("query")
                print(f"🔎 Searching for: {query}")
                function_response = search_tools.search(query)
            
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": function_response
            })
        
        response = client.chat.completions.create(
            model="gpt-4",
            messages=messages,
            tools=search_agent_tools,
            tool_choice="auto"
        )
        response_message = response.choices[0].message
        messages.append(response_message)
    
    return response_message.content

In [62]:
print("=== Indexing Main Capybara Page ===")
result = run_search_agent("Index this page: https://en.wikipedia.org/wiki/Capybara")
print(result)
print("\n" + "="*50 + "\n")

=== Indexing Main Capybara Page ===
📚 Indexing page: https://en.wikipedia.org/wiki/Capybara
✅ Successfully indexed: https://en.wikipedia.org/wiki/Capybara
The page "https://en.wikipedia.org/wiki/Capybara" has been successfully indexed. If you have any questions or need information about capybaras, you can now ask, and I will search the indexed page for it.




In [63]:
pages = [
    "https://en.wikipedia.org/wiki/Lesser_capybara",
    "https://en.wikipedia.org/wiki/Hydrochoerus",
    "https://en.wikipedia.org/wiki/Neochoerus",
    "https://en.wikipedia.org/wiki/Caviodon",
    "https://en.wikipedia.org/wiki/Neochoerus_aesopi"
]

for page in pages:
    print(f"Indexing: {page}")
    result = run_search_agent(f"Index this page: {page}")
    print(result)
    print()

Indexing: https://en.wikipedia.org/wiki/Lesser_capybara
📚 Indexing page: https://en.wikipedia.org/wiki/Lesser_capybara
✅ Successfully indexed: https://en.wikipedia.org/wiki/Lesser_capybara
The page "https://en.wikipedia.org/wiki/Lesser_capybara" has been indexed successfully. You can now search this page along with others that are already indexed.

Indexing: https://en.wikipedia.org/wiki/Hydrochoerus
📚 Indexing page: https://en.wikipedia.org/wiki/Hydrochoerus
✅ Successfully indexed: https://en.wikipedia.org/wiki/Hydrochoerus
The page https://en.wikipedia.org/wiki/Hydrochoerus has been successfully indexed. What do you want to do next?

Indexing: https://en.wikipedia.org/wiki/Neochoerus
📚 Indexing page: https://en.wikipedia.org/wiki/Neochoerus
✅ Successfully indexed: https://en.wikipedia.org/wiki/Neochoerus
The page https://en.wikipedia.org/wiki/Neochoerus has been indexed successfully. You may now search across all indexed pages.

Indexing: https://en.wikipedia.org/wiki/Caviodon
📚 Inde

In [64]:
print(f"Total pages indexed: {len(search_tools.indexed_pages)}")
print("Indexed URLs:")
for url in search_tools.indexed_pages.keys():
    print(f"  - {url}")
print("\n" + "="*50 + "\n")

Total pages indexed: 6
Indexed URLs:
  - https://en.wikipedia.org/wiki/Capybara
  - https://en.wikipedia.org/wiki/Lesser_capybara
  - https://en.wikipedia.org/wiki/Hydrochoerus
  - https://en.wikipedia.org/wiki/Neochoerus
  - https://en.wikipedia.org/wiki/Caviodon
  - https://en.wikipedia.org/wiki/Neochoerus_aesopi




In [65]:
print("=== Searching for Threats ===")

=== Searching for Threats ===


In [66]:
result = run_search_agent("What are threats to capybara populations?")

🔎 Searching for: threats to capybara populations


In [67]:
print(result)

Capybaras have flourished in cattle ranches and they roam in home ranges averaging 10 hectares (25 acres) in high-density populations. There may be potential risks due to human interactions, however, the exact threats to capybara populations aren't explicitly outlined in the indexed pages. For comprehensive information, you might want to look at more specialized sources or research studies. [Wikipedia - Capybara](https://en.wikipedia.org/wiki/Capybara)
